In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn import preprocessing
import skimage
from scipy.stats import skew, kurtosis
from sklearn import svm
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn import metrics
from skimage.feature import local_binary_pattern
from dataset.dataset import SkinLesion_Dataset, SegExamples
from pipeline.preprocessing import SkinLesionPreprocessing
from sklearn.model_selection import StratifiedKFold
from statistics import stdev
from sklearn import linear_model
from tqdm import tqdm


preproc = SkinLesionPreprocessing()
examples = SegExamples()

# get features example
all_feat = []
labels = []

radius = 1
n_points = 8 * radius
lbp_feature_vector=[]

for i in tqdm(range(examples.seg_examples_df.shape[0]-67)):

    if examples[i]['problem'] != 'binary':
        continue

    labels.append(examples[i]['label'])

    image = examples[i]['img']

    image = preproc.preprocess(image) # the img uploaded as numpy array.

    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Blur using 3 * 3 kernel.
    gray_blurred = cv2.blur(gray, (3, 3))

    cimg = cv2.cvtColor(gray_blurred,cv2.COLOR_GRAY2BGR)
    circles = cv2.HoughCircles(gray_blurred,cv2.HOUGH_GRADIENT,1,20,
                                param1=50,param2=30,minRadius=0,maxRadius=0)

    # circles = np.uint16(np.around(circles))
    # for i in circles[0,:]:
    #     # draw the outer circle
    #     cv2.circle(cimg,(i[0],i[1]),i[2],(0,255,0),2)
    #     # draw the center of the circle
    #     cv2.circle(cimg,(i[0],i[1]),2,(0,0,255),3)
    # cv2.imshow('detected circles',cimg)
    # cv2.waitKey(0)
    # cv2.destroyAllWindows()

    # Draw circles that are detected.
    if circles is not None:

        # Convert the circle parameters a, b and r to integers.
        detected_circles = np.uint16(np.around(circles))

        for pt in detected_circles[0, :]:
            a, b, r = pt[0], pt[1], pt[2]

            # Draw the circumference of the circle.
            cv2.circle(cimg, (a, b), r, (0, 255, 0), 2)

            # Draw a small circle (of radius 1) to show the center.
            cv2.circle(cimg, (a, b), 1, (0, 0, 255), 3)

        cv2.imshow("Detected Circle", cimg)
        cv2.waitKey(0)


100%|██████████| 1/1 [00:01<00:00,  1.99s/it]
